# **Case Técnico – Cientista de Dados Júnior | Datarisk (Entendimento dos dados)**

Nesta primeira etapa, realiza-se uma análise detalhada das bases fornecidas com o objetivo de compreender sua estrutura, granularidade, relacionamentos e limitações. Esse passo é fundamental para estabelecer os alicerces da modelagem e garantir que as transformações posteriores sejam consistentes com a natureza temporal e operacional do problema.

O foco principal deste notebook inclui:

* **Leitura e inspeção das bases originais:**
  Avaliam-se tipos de dados, distribuições iniciais, presença de valores ausentes e possíveis inconsistências.

* **Análise da granularidade por base:**

  * *base_cadastral:* informações estáticas por cliente.
  * *base_info:* informações mensais (cliente × safra).
  * *base_pagamentos_desenvolvimento:* registros transacionais de cobranças com data de pagamento.

* **Identificação das chaves de junção:**
  São analisadas as combinações possíveis entre as bases para garantir merges consistentes (incluindo `ID_CLIENTE`, `SAFRA_REF` e o relacionamento transacional).

* **Avaliação inicial da temporalidade:**
  As datas são inspecionadas para facilitar a construção do target, a engenharia de features e o respeito à ordem temporal durante a modelagem.

Essa etapa fornece a visão geral necessária para verificar a completude, coerência e utilidade das variáveis, além de orientar as futuras transformações e escolhas de modelagem.



### **1. Importação das bibliotecas**

**Bibliotecas utilizadas:**
- pandas: Manipulação de tabelas
- numpy: Manipulação numérica
- warnings: Evitar avisos indesejados
- os: Manipular diretórios
- pathlib: Manipulação de caminhos

In [ ]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

from warnings import filterwarnings

from src.eda_utils import *

# Ignorar avisos
filterwarnings('ignore')

# Configs pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

os.makedirs('../data/processed', exist_ok=True)

### **2. base_cadastral.csv**

**Leitura do dataset**

In [ ]:
base_cadastral = read_data('base_cadastral.csv', info=True)

**Sobre a base**
- A base cadastral possui informações a respeito do cadastro realizado dos clientes. Dessa forma, nela estão contidas informações essenciais sobre o cliente.
- Ao todo, a base possui 1315 registros e 8 colunas


**Sobre as colunas**
- ID_CLIENTE: 
    - Id único utilizado para cadastrar os clientes; 
    - Sem valores nulos;
    - 1315 clientes identificados.
- DATA CADASTRO:
    - Data de cadastro do cliente;
    - Sem valores nulos;
    - 777 datas identificadas.
- DDD:
    - Identificador de Discagem Direta à Distância;
    - 237 valores nulos encontrados, representam 14,45% dos registros presentes;
    - 79 valores identificados, alguns estão com erros (provalvemente de inserção na tabela, pois começam com "(");
    - Presença de DDDs inválidos, como o 52 e o 00.
- FLAG_PF:
    - Indica se o cliente é pessoa física ou não;
    - Valores nulos identificados, entretanto, essa ausência significa que o cliente **não é** uma pessoa física;
    - 1 Valor único identificado ('X' em caso afirmativo do cliente ser pessoa física).
- SEGMENTO_INDUSTRIAL:
    -Identifica qual segmento o cliente atua;
    - 83 valores nulos identificados, representando 5,06% dos registros;
    - 3 valores únicos identificados.
- DOMINIO_EMAIL:
    - Identifica qual o domínio do e-mail utilizado para cadastro do cliente;
    - 30 valores nulos identificados, representando 1,83% dos registros;
    - 6 valores únicos identificados.
- PORTE:
    - Identifica o porte do cliente;
    - 41 valores nulos encontrados, representando cerca de 2,50% dos registros;
    - 3 valores únicos identificados.
- CEP_2_DIG:
    - Indica os dois primeiros dígitos do CEP do cliente;
    - Valores nulos identificados, porém não parametrizados para serem capturados via método '.isna()';
    - 90 valores únicos identificados

**Próximos passos**
- Para garantir a integridade da modelagem, bem como da ánalise, as seguintes correções/alterações serão feitas nas colunas:
    1. Mapeamento da coluna FLAG_PF, transformando 'NaN's em 0 e 'X' em 1
    2. Alteração da coluna DATA_CADASTRO para datetime pandas
    3. Entedimento sobre DDDs incongruentes ou com erros
    4. Correção de NaNs na coluna CEP_2_DIG


In [ ]:
base_cadastral['FLAG_PF'].fillna(0, inplace=True)
base_cadastral.loc[base_cadastral['FLAG_PF'] == 'X', 'FLAG_PF'] = 1

base_cadastral['FLAG_PF'].unique()

In [ ]:
base_cadastral['DATA_CADASTRO'] = pd.to_datetime(base_cadastral['DATA_CADASTRO'])

In [ ]:
base_cadastral.info()

In [ ]:
not_available_ddds = ['20', '23', '25', '26', '29', '30', '36', '39', '40', '50', '56', '57', '58', '59', '60', '70', '72', '76', '78', '80', '90', '00']

# Filtro
inconsistences_registers_bcs = base_cadastral.loc[(base_cadastral['DDD'].isin(not_available_ddds)) | (base_cadastral['DDD'].str.startswith('(')), :]

print('Quantidade de registros incongruentes presentes:')
print(inconsistences_registers_bcs.shape[0])
print()
inconsistences_registers_bcs

In [ ]:
mask_ddd_bc = base_cadastral['DDD'].isin(not_available_ddds) | base_cadastral['DDD'].str.startswith('(')
base_cadastral.loc[mask_ddd_bc, 'DDD'] = 'INVÁLIDO'

base_cadastral.loc[base_cadastral['DDD'] == mask_ddd_bc]

Optarei por não excluir registros por hora, essa decisão será tomada durante a fase de EDA do processo, dessa forma, devemos marcar estas incongruências com uma flag. Portanto, estes registros terão seus DDDs alterados para 'INVÁLIDO'.

In [ ]:
base_cadastral.loc[base_cadastral['CEP_2_DIG'] == 'na', :]

In [ ]:
base_cadastral.loc[base_cadastral['CEP_2_DIG'] == 'na', 'CEP_2_DIG'] = np.nan

In [ ]:
base_cadastral['DDD'].str.startswith('(').sum()

### **3. base_info.csv**

**Leitura do dataset**

In [ ]:
base_info = read_data('base_info.csv', info=True)

**Sobre a base**
- A base info tem o intuito de compor informações adicionais a respeito dos clientes. Ela é atualizada mensalmente, e, cada cliente só aparece uma vez por mês (SAFRA_REF).
- A base possui 24401 registros e 4 colunas


**Sobre as colunas**
- ID_CLIENTE:
    - Mesma função da 'base_cadastral';
    - Sem valores nulos identificados;
    - 1316 valores únicos identificados, o que sugere uma incongruência em relação a 'base_cadastral' que possui 1315 valores únicos.
- SAFRA_REF:
    - Mesma função da 'base_cadastral';
    - Sem valores nulos identificados;
    - 40 valores únicos identificados. Como SAFRA_REF determina os meses em que estão sendo observados os registros, podemos assumir que existe um intervalo de 40 meses entre os dados, ou, 3 anos e 4 meses.
- RENDA_MES_ANTERIOR:
    - Identifica a renda ou o faturamento declarado pelo cliente no mês anterior;
    - 717 valores nulos identificados, representando 2,94% de todos os registros;
    - 23196 valores únicos identificados.
- NO_FUNCIONARIOS:
    - Número de funcionários declarado pelo cliente no mês anterior;
    - 1252 valores nulos identificados, representando 5,13% de todos os registros;
    - 128 valores únicos identificados.

**Próximos passos**
- Para garantir a integridade da modelagem, bem como da ánalise, as seguintes correções/alterações serão feitas nas colunas:
    1. Mudança de tipo de SAFRA_REF para datetime pandas


In [ ]:
base_info['SAFRA_REF'] = pd.to_datetime(base_info['SAFRA_REF'])

In [ ]:
base_info[['RENDA_MES_ANTERIOR', 'NO_FUNCIONARIOS']].describe().T

Alguns insights:
1. RENDA_MES_ANTERIOR -> provavelmente apresentará uma distribuição assimétrica, com cauda alongada para a direita. Isso é denotado tanto pelos valores observados no Q1 e Q3, quanto no desvio padrão
2. NO_FUNCIONARIOS -> Apresentará uma distribuição normal, em torno de 118 funcionários como média.

Vamos verificar qual ID está presente aqui e não está em 'base_cadastral'

In [ ]:
missing_id = base_info.loc[~(base_info['ID_CLIENTE'].isin(base_cadastral['ID_CLIENTE']))]

missing_clients = missing_id['ID_CLIENTE'].unique()
len(missing_clients), missing_clients[:10]

Optarei por remover estes clientes, visto que 'base_cadastral' é a referência de cadastro de clientes, portanto, ambas as bases, devem ser correspondentes.

In [ ]:
base_info = base_info[~base_info['ID_CLIENTE'].isin(missing_clients)]


base_info.loc[base_info['ID_CLIENTE'].isin(missing_clients)]

In [ ]:
base_info.shape

### **4. base_pagamentos_desenvolvimento.csv**

**Leitura do dataset**

In [ ]:
base_pag = read_data('base_pagamentos_desenvolvimento.csv', info=True)

**Sobre a base**
- A base pagamentos mostra como são os pagamentos dos clientes presentes;
- A base possui 77414 registros e 7 colunas.

**Sobre as colunas**
- ID_CLIENTE:
    - Mesma função da 'base_cadastral';
    - Sem valores nulos identificados;
    - 1248 valores únicos identificados, o que sugere uma incongruência em relação a 'base_cadastral' que possui 1315 valores únicos.
- SAFRA_REF:
    - Mesma função da 'base_cadastral';
    - Sem valores nulos identificados;
    - 35 valores únicos identificados. Como SAFRA_REF determina os meses em que estão sendo observados os registros, podemos assumir que existe um intervalo de 35 meses entre os dados, ou, 2 anos e 11 meses.
- DATA_EMISSAO_DOCUMENTO:
    - Identifica o dia em que foi realizado o pagamento da cobrança;
    - Sem valores nulos identificados;
    - 921 valores únicos identificados.
- DATA_VENCIMENTO:
    - Identifica a data limite para realizar o pagamento da cobrança;
    - Sem valores nulos identificados;
    - 955 valores únicos identificados.
- VALOR_A_PAGAR:
    - Identifica o valor a ser pago na cobrança;
    - 1170 valores nulos identificados, representando 1,51% de todos os registros;
    - 67588 valores únicos identificados.
- TAXA:
    - Identifica a taxa de juros cobrada no empréstimo;
    - Sem valores nulos identificados;
    - 5 valores únicos identificados (forte evidência de ser uma variável categórica, pois "segmenta" clientes)


**Próximos passos**
- Para garantir a integridade da modelagem, bem como da ánalise, as seguintes correções/alterações serão feitas nas colunas:
    1. Mudança de tipo de SAFRA_REF para datetime pandas
    2. Check se todos os ID_CLIENTE presentes batem com os da 'base_cadastral'


In [ ]:
base_pag.describe()

In [ ]:
datetime_cols = ['SAFRA_REF', 'DATA_EMISSAO_DOCUMENTO', 'DATA_PAGAMENTO', 'DATA_VENCIMENTO']

base_pag[datetime_cols] = base_pag[datetime_cols].apply(pd.to_datetime)

In [ ]:
missing_id = base_pag.loc[~(base_pag['ID_CLIENTE'].isin(base_cadastral['ID_CLIENTE']))]

missing_clients = missing_id['ID_CLIENTE'].unique()
len(missing_clients), missing_clients[:10]

Isso denota que a diferença de registros apenas mostra que existem clientes cadastrados que não possuem movimentações registradas no período de tempo observado. Além disso, optarei por hora, não excluir ou inserir valores em registros que possuem valores faltantes na coluna VALOR_A_PAGAR com o intuito de evitar data leakeage. Essa inserção/remoção ocorrerá durante a EDA.




**Processamento das bases**


Para preservas as mudanças realizadas nas colunas, bem como nas bases, optarei por transformá-las em um arquivo parquet (.parquet) para que estas mudanças perpetuem durante toda a análise e modelagem.
 

In [ ]:
base_cadastral.to_parquet('../data/processed/base_cadastral.parquet', index=False)
base_info.to_parquet('../data/processed/base_info.parquet', index=False)
base_pag.to_parquet('../data/processed/base_pagamentos_desenvolvimento.parquet', index=False)